<a href="https://colab.research.google.com/github/DinurakshanRavichandran/Visio-Glance/blob/Fundus-eye-disease-detection/custom_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, classification_report

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Constants
IMG_SIZE = 224
CATEGORIES = ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']
BATCH_SIZE = 32
EPOCHS = 50
DATA_DIR = '/content/drive/MyDrive/data set/dataset'

# ================== Data Preprocessing ==================
def dual_preprocess(image):
    """Process single image for both models"""
    image = tf.cast(image, tf.float32)
    return {
        'vgg_input': vgg_preprocess(image),
        'resnet_input': resnet_preprocess(image)
    }

# Create base generator without preprocessing
base_datagen = ImageDataGenerator(validation_split=0.2)

# Train generator
train_generator = base_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    classes=CATEGORIES
)

# Validation generator
val_generator = base_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    classes=CATEGORIES
)

# ================== Dual Input Pipeline ==================
def dual_input_generator(generator):
    for x, y in generator:
        # Create separate preprocessing for each model
        x_vgg = vgg_preprocess(x.copy())  # BGR + mean subtraction
        x_resnet = resnet_preprocess(x.copy())  # Scaling to [-1, 1]
        yield ((x_vgg, x_resnet), y)

# Create TensorFlow datasets
train_dual_dataset = tf.data.Dataset.from_generator(
    lambda: dual_input_generator(train_generator),
    output_signature=(
        (
            tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32)
        ),
        tf.TensorSpec(shape=(None, len(CATEGORIES)), dtype=tf.float32)
    )
).prefetch(tf.data.AUTOTUNE)

val_dual_dataset = tf.data.Dataset.from_generator(
    lambda: dual_input_generator(val_generator),
    output_signature=(
        (
            tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32)
        ),
        tf.TensorSpec(shape=(None, len(CATEGORIES)), dtype=tf.float32)
    )
).prefetch(tf.data.AUTOTUNE)

# ================== Enhanced Model Architecture ==================
def create_model():
    # Base models with proper input shapes
    vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

    # Freeze base models
    for layer in vgg_base.layers:
        layer.trainable = False
    for layer in resnet_base.layers:
        layer.trainable = False

    # Feature extraction
    vgg_features = GlobalAveragePooling2D()(vgg_base.output)
    resnet_features = GlobalAveragePooling2D()(resnet_base.output)
    merged = tf.keras.layers.concatenate([vgg_features, resnet_features])

    # Enhanced classifier with preprocessing-aware architecture
    x = Dense(512, activation='relu', kernel_regularizer=l2(0.02))(merged)
    x = BatchNormalization()(x)
    x = Dropout(0.7)(x)
    outputs = Dense(len(CATEGORIES), activation='softmax')(x)

    return Model(inputs=[vgg_base.input, resnet_base.input], outputs=outputs)

model = create_model()

# ================== Class Weighting ==================
y_train = train_generator.classes
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {i: class_weights[i] for i in range(len(CATEGORIES))}

# ================== Model Compilation ==================
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    weighted_metrics=['accuracy']
)

# ================== Training Configuration ==================
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
]

# ================== Model Training ==================
history = model.fit(
    train_dual_dataset,
    validation_data=val_dual_dataset,
    epochs=EPOCHS,
    callbacks=callbacks,
    class_weight=class_weight_dict,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

# ================== Post-Training Analysis ==================
# Load best model for evaluation
best_model = tf.keras.models.load_model('best_model.h5')

# Generate predictions
y_true = val_generator.classes
y_pred_probs = best_model.predict(val_dual_dataset, steps=len(val_generator))
y_pred = np.argmax(y_pred_probs, axis=1)[:len(y_true)]  # Trim to match exact size

# Generate reports
print("\nEnhanced Classification Report:")
print(classification_report(y_true, y_pred, target_names=CATEGORIES, digits=4))

print("\nConfusion Matrix:")
print(tf.math.confusion_matrix(y_true, y_pred))

# Save final model
best_model.save('/content/retinal_disease_final_model.keras')
print("\nModel saved with complete preprocessing pipeline!")

Mounted at /content/drive
Found 3384 images belonging to 4 classes.
Found 844 images belonging to 4 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.5295 - loss: 17.4485 - weighted_accuracy: 0.5282 

106/106 ━━━━━━━━━━━━━━━━━━━━ 1494s 13s/step - accuracy: 0.5307 - loss: 17.4351 - weighted_accuracy: 0.5294 - val_accuracy: 0.7204 - val_loss: 13.8142 - val_weighted_accuracy: 0.7204 - learning_rate: 1.0000e-04
Epoch 2/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 409ms/step - accuracy: 0.7900 - loss: 12.8298 - weighted_accuracy: 0.7889

106/106 ━━━━━━━━━━━━━━━━━━━━ 52s 496ms/step - accuracy: 0.7901 - loss: 12.8224 - weighted_accuracy: 0.7890 - val_accuracy: 0.7536 - val_loss: 10.7458 - val_weighted_accuracy: 0.7536 - learning_rate: 1.0000e-04
Epoch 3/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step - accuracy: 0.8403 - loss: 9.9617 - weighted_accuracy: 0.8386

106/106 ━━━━━━━━━━━━━━━━━━━━ 52s 496ms/step - accuracy: 0.8402 - loss: 9.9567 - weighted_accuracy: 0.8386 - val_accuracy: 0.7239 - val_loss: 8.8224 - val_weighted_accuracy: 0.7239 - learning_rate: 1.0000e-04
Epoch 4/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step - accuracy: 0.8520 - loss: 8.0020 - weighted_accuracy: 0.8505

106/106 ━━━━━━━━━━━━━━━━━━━━ 78s 743ms/step - accuracy: 0.8521 - loss: 7.9982 - weighted_accuracy: 0.8505 - val_accuracy: 0.7725 - val_loss: 7.1602 - val_weighted_accuracy: 0.7725 - learning_rate: 1.0000e-04
Epoch 5/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step - accuracy: 0.8520 - loss: 6.6015 - weighted_accuracy: 0.8502

...

106/106 ━━━━━━━━━━━━━━━━━━━━ 42s 397ms/step - accuracy: 0.9282 - loss: 0.7650 - weighted_accuracy: 0.9271 - val_accuracy: 0.8282 - val_loss: 1.0362 - val_weighted_accuracy: 0.8282 - learning_rate: 1.0000e-04
Epoch 32/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 79s 745ms/step - accuracy: 0.9310 - loss: 0.7366 - weighted_accuracy: 0.9296 - val_accuracy: 0.7903 - val_loss: 1.1997 - val_weighted_accuracy: 0.7903 - learning_rate: 1.0000e-04
Epoch 33/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 42s 397ms/step - accuracy: 0.9250 - loss: 0.7421 - weighted_accuracy: 0.9238 - val_accuracy: 0.7844 - val_loss: 1.0984 - val_weighted_accuracy: 0.7844 - learning_rate: 1.0000e-04
Epoch 34/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - accuracy: 0.9365 - loss: 0.6707 - weighted_accuracy: 0.9353

106/106 ━━━━━━━━━━━━━━━━━━━━ 44s 416ms/step - accuracy: 0.9365 - loss: 0.6707 - weighted_accuracy: 0.9353 - val_accuracy: 0.8400 - val_loss: 0.9018 - val_weighted_accuracy: 0.8400 - learning_rate: 1.0000e-04
Epoch 35/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 79s 745ms/step - accuracy: 0.9349 - loss: 0.6521 - weighted_accuracy: 0.9338 - val_accuracy: 0.7832 - val_loss: 1.0422 - val_weighted_accuracy: 0.7832 - learning_rate: 1.0000e-04
Epoch 36/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 43s 404ms/step - accuracy: 0.9455 - loss: 0.6088 - weighted_accuracy: 0.9445 - val_accuracy: 0.8329 - val_loss: 0.9161 - val_weighted_accuracy: 0.8329 - learning_rate: 1.0000e-04
Epoch 37/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - accuracy: 0.9435 - loss: 0.5968 - weighted_accuracy: 0.9427

106/106 ━━━━━━━━━━━━━━━━━━━━ 40s 376ms/step - accuracy: 0.9434 - loss: 0.5968 - weighted_accuracy: 0.9426 - val_accuracy: 0.8389 - val_loss: 0.8483 - val_weighted_accuracy: 0.8389 - learning_rate: 1.0000e-04
Epoch 38/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 78s 739ms/step - accuracy: 0.9488 - loss: 0.5684 - weighted_accuracy: 0.9478 - val_accuracy: 0.7666 - val_loss: 1.0421 - val_weighted_accuracy: 0.7666 - learning_rate: 1.0000e-04
Epoch 39/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 43s 410ms/step - accuracy: 0.9419 - loss: 0.5549 - weighted_accuracy: 0.9407 - val_accuracy: 0.8009 - val_loss: 1.0007 - val_weighted_accuracy: 0.8009 - learning_rate: 1.0000e-04
Epoch 40/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 41s 387ms/step - accuracy: 0.9373 - loss: 0.5542 - weighted_accuracy: 0.9362 - val_accuracy: 0.8164 - val_loss: 0.9036 - val_weighted_accuracy: 0.8164 - learning_rate: 1.0000e-04
Epoch 41/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step - accuracy: 0.9439 - loss: 0.5367 - weighted_accuracy: 0.9430

106/106 ━━━━━━━━━━━━━━━━━━━━ 80s 761ms/step - accuracy: 0.9438 - loss: 0.5367 - weighted_accuracy: 0.9429 - val_accuracy: 0.8637 - val_loss: 0.6984 - val_weighted_accuracy: 0.8637 - learning_rate: 1.0000e-04
Epoch 42/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 42s 400ms/step - accuracy: 0.9353 - loss: 0.5317 - weighted_accuracy: 0.9341 - val_accuracy: 0.8140 - val_loss: 0.8703 - val_weighted_accuracy: 0.8140 - learning_rate: 1.0000e-04
Epoch 43/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 41s 386ms/step - accuracy: 0.9418 - loss: 0.5196 - weighted_accuracy: 0.9406 - val_accuracy: 0.7784 - val_loss: 1.0002 - val_weighted_accuracy: 0.7784 - learning_rate: 1.0000e-04
Epoch 44/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 80s 761ms/step - accuracy: 0.9392 - loss: 0.4920 - weighted_accuracy: 0.9383 - val_accuracy: 0.7891 - val_loss: 0.9036 - val_weighted_accuracy: 0.7891 - learning_rate: 1.0000e-04
Epoch 45/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 43s 407ms/step - accuracy: 0.9462 - loss: 0.4704 - weighted_accuracy: 0.9450 - val_accuracy

27/27 ━━━━━━━━━━━━━━━━━━━━ 16s 432ms/step

Enhanced Classification Report:
                      precision    recall  f1-score   support

            cataract     0.2876    0.3140    0.3002       207
diabetic_retinopathy     0.2546    0.2489    0.2517       221
            glaucoma     0.2406    0.2228    0.2314       202
              normal     0.3070    0.3084    0.3077       214

            accuracy                         0.2737       844
           macro avg     0.2725    0.2735    0.2728       844
        weighted avg     0.2726    0.2737    0.2729       844


Confusion Matrix:
tf.Tensor(
[[65 55 43 44]
 [50 55 55 61]
 [59 54 45 44]
 [52 52 44 66]], shape=(4, 4), dtype=int32)

Model saved with complete preprocessing pipeline!
